# Jupyter / IPython Notebooks # 

Sommersemester 2016, Lars Kaleschke

[Jupyter Notebook](http://jupyter.org/) ist eine webbasierte Anwendung, die 2014 aus IPython entstanden ist. Sie erlaubt die Kombination von Sourcecode, Gleichungen, Visualisierungen und wissenschaftlicher Dokumentation in einem Dokument.

Für die meisten Übungen reicht die Cloud-Umgebung https://cloud.sagemath.com/ völlig aus. Im Folgenden wird beschrieben, wie eine Jupyter-Server im ZMAW-Netz sicher gestartet und genutzt werden kann.


## Konfiguration im ZMAW-Netz

Zunächst loggen wir uns auf dem Rechner squall3 ein

    ssh -X squall3

Dann laden wir die Umgebungsvariablen 

    module load matlab/2015b
    module load python3/3.5.0

Um diese module load Befehle bei jedem neuen Einloggen nicht immer wieder neu eingeben zu müssen, können diese in der versteckten Datei ~.profile eingetragen werden.

Wir installieren einen zusätzlichen Matlab-Kernel für die Jupyter-Umgebung 

    python3 -m matlab_kernel.install

Um mit dem Jupyter-System zu arbeiten, muss zunächst ein Serverprozess gestartet werden. Es ist sehr anzuraten, dass der Jupyter-Server mit einem Passwort geschützt wird. Dazu erstellen wir die entsprechende Konfigurationsdatei mit einem kleinen Skript (siehe unten)
    
    python3 jupyterconfig.py

Nach Eingabe eines Passwortes ist der Jupyter-Server (hoffentlich) einsatzbereit. Das Passwort wird nicht im Klartext, sondern als Hash verschlüsselt in der Konfigurationsdatei gespeichert. Der Jupyter-Notebook-Server wird dann gestartet mit dem Befehl

    jupyter notebook &

Es sollten nun einige Ausgaben auf dem Terminal zu finden sein, unter anderem

    The Jupyter Notebook is running at: http://localhost:8888/
    Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).

Es sollte sich auch ein Browser (Firefox) öffnen und ein Passwort angefragt werden. Falls schon ein weiterer Firefox-Prozess auf einem anderen Rechner laufen sollte, so ist dieser zunächst zu stoppen. Nach dem Start von Firefox auf squall3 ist die obige URL (http://localhost:8888/) in den Browser einzugeben, wobei 8888 durch die entsprechende Portnummer ersetzt werden muss.

Der Jupyter Prozess läuft im Hintergrund weiter, auch wenn Sie sich ausloggen. Um den Prozess zu stoppen, können Sie ihn entweder auf der Unix-Terminalkonsole mit

    fg

in den Vordergrund holen und dann mit der Tastenkombination Strg/CTRL C abbrechen. Eine weitere Methode ist folgender Befehl

    killall ipython    

## Konfigurationshilfe 

In [1]:
%%file jupyterconfig.py
import os,os.path
from notebook.auth import passwd
from pylab import loadtxt

home=os.getenv('HOME')
cfile=home+'/.jupyter/jupyter_notebook_config.py'

if os.path.exists(cfile):
    print(os.path.basename(cfile),'exists')
    T=loadtxt(cfile,comments='#',dtype=str)
else:
    print('Generating config file')
    os.system('jupyter notebook --generate-config')
    T=loadtxt(cfile,comments='#',dtype=str)

has_passwd=False
for t in T:
    if str(t).find('c.NotebookApp.password')!=-1:
        has_passwd=True
        print(os.path.basename(cfile),'includes password hash')
        
if has_passwd==False:
    print('No password hash found')
    sha=passwd()
    l1='c.NotebookApp.password = "'+sha+'"\n'
    l2='c.NotebookApp.browser = "firefox"\n'
    fid=open(cfile,'a')
    fid.write(l1)
    fid.write(l2)
    fid.close()
    print('Added password hash to configuration file')
    print('Please restart jupyter server')

Overwriting jupyterconfig.py
